In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def append_ext(fn):
    return fn + ".jpg"
train_df = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv', dtype = str)
test_df = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv', dtype = str)
train_df["image_name"] = train_df["image_name"].apply(append_ext)
test_df["image_name"] = test_df["image_name"].apply(append_ext)


In [ ]:
datagen = ImageDataGenerator (rescale = 1./255.,
                             validation_split = 0.30,
                             rotation_range = 10,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             brightness_range = [0.2,1.2])
                              

In [ ]:
train_gen = datagen.flow_from_dataframe(
                  dataframe = train_df,
                  directory = '../input/planets-dataset/planet/planet/train-jpg',
                  subset = 'training',
                  batch_size = 300,
                  seed = 42,
                  shuffle = True,
                  class_mode = 'categorical',
                  target_size = (32,32),
                  x_col = 'image_name',
                  y_col = 'tags')

valid_gen = datagen.flow_from_dataframe(
                  dataframe = train_df,
                  directory = '../input/planets-dataset/planet/planet/train-jpg',
                  subset = 'validation',
                  batch_size = 300,
                  seed = 42,
                  shuffle = True,
                  class_mode = 'categorical',
                  target_size = (32, 32),
                  x_col = 'image_name',
                  y_col = 'tags')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.regularizers import l1_l2

Arranging model architecture starting with the input layer using VGGI6 implementation

In [ ]:
model = Sequential()

# input layer
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(2048))
model.add(Dense(2048))
model.add(Activation('relu'))
model.add(Dense(449, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
%%time
train_steps = train_gen.n//train_gen.batch_size
valid_steps = train_gen.n//train_gen.batch_size

n_epochs = 5
optimizer = optimizers.RMSprop(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
%%time
history = model.fit(train_gen,
                    epochs=n_epochs, 
                    steps_per_epoch=train_steps,
                    validation_data=valid_gen,
                    validation_steps=valid_steps)

In [ ]:
test_gen1 = datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = '../input/planets-dataset/planet/planet/test-jpg',
    x_col = 'image_name',
    y_col = None,
    batch_size = 300,
    seed = 42,
    shuffle = False,
    class_mode = None,
    target_size = (32, 32))

test_gen2 = datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = '../input/planets-dataset/test-jpg-additional/test-jpg-additional',
    x_col = 'image_name',
    y_col = None,
    batch_size = 300,
    seed = 42,
    shuffle = False,
    class_mode = None,
    target_size = (32, 32))


In [ ]:
test_steps = test_gen1.n//test_gen1.batch_size

In [ ]:
test_gen1.reset()
prediction = model.predict(test_gen1, steps= test_steps, verbose=1)


In [ ]:
test_gen2.reset()
prediction2 = model.predict(test_gen2, steps= test_steps, verbose=1)


In [ ]:

predicted_class_indices = np.argmax(prediction, axis=1)
predicted_class_indices2 = np.argmax(prediction2, axis=1)

labels = (train_gen.class_indices)
labels = dict((v, k) for k, v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames = test_gen1.filenames
predict = pd.DataFrame(predictions, columns=['tags'])
filename = pd.DataFrame(filenames, columns=['image_name'])[:40460]

In [ ]:
labels2 = (train_gen.class_indices)
labels2 = dict((v, k) for k, v in labels2.items())
predictions2 = [labels2[k] for k in predicted_class_indices2]

filenames = test_gen2.filenames
predict2 = pd.DataFrame(predictions2, columns=['tags'])
filename2 = pd.DataFrame(filenames, columns=['image_name'])

In [ ]:
results = pd.concat([filename, predict], axis=1)
results = pd.concat([filename, predict], axis=1)
results2 = pd.concat([filename2, predict2], axis=1)
final_result = pd.concat([results, results2])

In [ ]:
final_result
final_result.to_csv("./final-result.csv", index=False)